In [30]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.mobilenet_v2 import preprocess_input
from keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
# from google.colab.patches import cv2_imshow

In [31]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [32]:
# 경로 수정할 것
# face_dnn_model_name = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/res10_300x300_ssd_iter_140000.caffemodel'
# prototxt_name = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/deploy.prototxt.txt'

face_dnn_model_name = 'C:/mask/res10_300x300_ssd_iter_140000.caffemodel'
prototxt_name = 'C:/mask/deploy.prototxt.txt'

In [33]:
# 변수 설정
min_confidence = 0.3 
frame_width = 700 # 열린 창의 크기

In [34]:
# 모델 객체 생성
face_detect_model = cv2.dnn.readNetFromCaffe(prototxt_name, face_dnn_model_name)
# mask_detect_model = load_model('/content/gdrive/MyDrive/Project/Mask_Detection_Model/mask_detector.model')
mask_detect_model = load_model('C:/mask/mask_detector.model')

In [35]:
def detectAndDisplay(frame, type):
    # frame_width 에 맞춰서 image resize - 화면에 영상을 띄울때 크기설정
    (height, width) = frame.shape[:2]
    ratio = frame_width / width
    dimension = (frame_width, int(height * ratio))
    frame = cv2.resize(frame, dimension, interpolation = cv2.INTER_AREA)

    # 이미지를 300 x 300으로 size 조정하고 blob 만들기
    # blob = cv2.dnn.blobFromImage(img, scalefactor=1., size=(300, 300), mean=(104., 177., 123.))
    # 
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), scalefactor=1.0,
                                 size=(300, 300), mean=(104.0, 177.0, 123.0))
    
    # blob 모델에 넣고 얼굴 detection 수행
    face_detect_model.setInput(blob)
    face_detections = face_detect_model.forward()

    result_frame = frame.copy()

    # Detections 한 수만큼 루프 돌기
    for i in range(0, face_detections.shape[2]):
        confidence = face_detections[0, 0, i, 2] # confidence는 detection한 확률

        # min_confidence 보다 큰 경우에만 detection으로 인정함
        if confidence > min_confidence:
            print('Face Detection Complete')
            # box 좌표 구하기
            box = face_detections[0, 0, i, 3:7] * np.array([frame_width, int(height * ratio), frame_width, int(height * ratio)])
            (x1, y1, x2, y2) = box.astype('int')
            
            # 얼굴 이미지 자르기
            face = frame[y1:y2, x1:x2]

            # 얼굴 이미지를 입력값에 맞게 변형
            face_input = cv2.resize(face, dsize=(224, 224))
            face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
            face_input = preprocess_input(face_input)
            face_input = np.expand_dims(face_input, axis=0)

            # 얼굴 이미지를 모델에 넣기
            mask, nomask = mask_detect_model.predict(face_input).squeeze()

            # 마스크 인식 결과 비교
            if mask > nomask:
                color = (0, 255, 0)
                label = 'Mask %d%%' % (mask * 100)
            else:
                color = (0, 0, 255)
                label = 'No Mask %d%%' % (nomask * 100)

            # 얼굴에 사각형 그리기
            cv2.rectangle(result_frame, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
            cv2.putText(result_frame, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=color, thickness=2, lineType=cv2.LINE_AA)

    # 결과 출력
    if type == 'vedio':
        global writer
        if writer is None:
            fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
            out = cv2.VideoWriter('output.mp4', fourcc, frame.get(cv2.CAP_PROP_FPS), (frame.shape[1], frame.shape[0]))
        
        # disk에 frame을 write
        if writer is not None:
            out.write(result_frame)
    
    # camera인 경우
    else:
        cv2.imshow('camera', result_frame)

In [41]:
# 동영상 입력(vedio), 카메라 입력(camera)
# cap = cv2.VideoCapture('path') # 비디오에 모델을 적용하고 싶은 경우
cap = cv2.VideoCapture(0) # 비디오 캡처 객체를 생성(캠을 위해 필수)

# cap.set(id, value)는 프로퍼티 변경 
# CAP_PROP_FRAME_WIDTH 프레임 폭
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)  # 프레임 폭을 현재 창의 크기로
# CAP_PROP_FRAME_HEIGHT프레임 높이
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_width) # 프레임 높이을 현재 창의 크기로

if cap.isOpened(): # 객체 초기화 확인 - 장치와 잘 연결되었는지 확인
    while True:
        
        # cap.read()는 영상 프레임 읽기 
        # ret는 프레임 읽기 성공 또는 실패 여부
        # img는 프레임 이미지, NumPy() 배열 또는 None()
        
        ret, frame = cap.read()
        
        if ret:
            detectAndDisplay(frame, 'cam')
            # waitKey() 는 일정 시간 지연 후 함수를 빠져 나옴
            #화면을 보는데 지연시간 필요 - 너무 빠르면 눈을 볼 수 없어서
            
            # 1ms 동안 키 입력 대기 - 아무 키나 눌렀으면 중지
            # 입력이 없으면 자동으로 -1이 들어감 -> -1이 아니면 키를 누른 것
            if cv2.waitKey(1) != -1: # if문에서 조건 설정
                break
        else:
            print('no frame!')

else:
    # 캠을 켤 수 없다는 안내
    print("can't open camera!")

cap.release() # 캡처 자원 반납
# writer.release()
cv2.destroyAllWindows() # 창 닫기

Face Detection Complete
1/1 [==============================] - 0s 259ms/step
Face Detection Complete
1/1 [==============================] - 0s 274ms/step
Face Detection Complete
1/1 [==============================] - 0s 236ms/step
Face Detection Complete
1/1 [==============================] - 0s 239ms/step
Face Detection Complete
1/1 [==============================] - 0s 268ms/step
Face Detection Complete
1/1 [==============================] - 0s 234ms/step
Face Detection Complete
1/1 [==============================] - 0s 220ms/step
Face Detection Complete
1/1 [==============================] - 0s 313ms/step
Face Detection Complete
1/1 [==============================] - 0s 296ms/step
Face Detection Complete
1/1 [==============================] - 0s 337ms/step
Face Detection Complete
1/1 [==============================] - 0s 349ms/step
Face Detection Complete
1/1 [==============================] - 0s 274ms/step
Face Detection Complete
1/1 [==============================] - 0s 336ms/step